# Library testing
This notebook tests the
[MadryLab/robustness](https://github.com/MadryLab/robustness)
package, and checks it
[can hook up to tensorboardX](https://cox.readthedocs.io/en/latest/examples/2.html).

### Import dependencies

In [1]:
import os
os.environ["NOTEBOOK_MODE"] = "1" # for robustness library

from robustness import model_utils, datasets, train, defaults
from robustness.datasets import CIFAR

# We use cox (http://github.com/MadryLab/cox) to log, store and analyze
# results. Read more at https//cox.readthedocs.io.
from cox.utils import Parameters
import cox.store

import torch
torch.cuda.get_device_name(0)

/home/ttw/.miniconda3/envs/robustness/lib/python3.6/site-packages/robustness/train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


'GeForce RTX 3080'

### Training setup

In [2]:
# Hard-coded dataset, architecture, batch size, workers
ds = CIFAR('/tmp/')
m, _ = model_utils.make_and_restore_model(
    arch='resnet50', dataset=ds
)
train_loader, val_loader = ds.make_loaders(
    batch_size=64, workers=24
)

# Create a cox store for logging
out_store = cox.store.Store("./cox")

# Hard-coded base parameters
train_kwargs = {
    'out_dir': "train_out",
    'adv_train': 1, # Use adversarial training
    'constraint': 'inf', # L-inf PGD
    'eps': 8.0 / 255.0, # Epsilon constraint (L-inf norm)
    'attack_lr': 1.5,
    'attack_steps': 20
}
train_args = Parameters(train_kwargs)

# Fill whatever parameters are missing from the defaults
train_args = defaults.check_and_fill_args(train_args,
                        defaults.TRAINING_ARGS, CIFAR)
train_args = defaults.check_and_fill_args(train_args,
                        defaults.PGD_ARGS, CIFAR)

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified
Logging in: /home/ttw/code/adversarial-ntks/notebooks/discard-hypothesis/cox/d7a17496-bc89-488e-8085-9e89a62a526f


### Train

In [ ]:
# Train a model
train.train_model(
    train_args,
    m,
    (train_loader, val_loader),
    store=out_store
)